# Notebook

In [2]:
#%matplotlib qt
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from mtflearn import ZPs

In [6]:
num_patches = 300
size = 45
shape = (num_patches, size, size)
# ps is a 3D numpy array
ps = np.random.random(shape)

zps = ZPs(n_max=10, size=size)
X = zps.fit_transform(ps)

In [9]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 3))
ax.plot(X[0])

In [ ]:
img_shape = (512, 512)
img = np.random.random(img_shape)

zps_size = 45
zps = ZPs(n_max=10, size=zps_size)
X = zps.fit_transform(img)